In [2]:
from datasets import load_dataset
import re
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Counter
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
ds = load_dataset("HugoGiddins/buy_sell_intent")
ds

DatasetDict({
    test: Dataset({
        features: ['post_id', 'subject_type', 'subject_identifier', 'subject_name', 'content', 'sarcasm', 'relevance', 'polarity', 'label_11', 'label_index', 'confidence'],
        num_rows: 940
    })
})

In [4]:
df = ds["test"].to_pandas()

In [5]:
df.shape

(940, 11)

In [6]:
df.iloc[1]

post_id                                                       real_7500
subject_type                                                      token
subject_identifier                                                   OP
subject_name                                                   Optimism
content               GM chat! ... 🧡 I want to start this day with s...
sarcasm                                                               0
relevance                                                             1
polarity                                                       positive
label_11                                                        bullish
label_index                                                           8
confidence                                                          0.9
Name: 1, dtype: object

**Cleaning the Text**

In [7]:
_CLEAN_RE = re.compile(
    r"""
    ^\d+\s*            # leading number like "0  "
    |\[INST\].*?\[/INST\]  # full [INST] ... [/INST] block (non-greedy)
    |\[/?INST\]        # any leftover [INST] or [/INST]
    """,
    flags=re.DOTALL | re.MULTILINE | re.VERBOSE
)

In [8]:
def clean_text(text):
    text = _CLEAN_RE.sub("", text)
    text = re.sub(r"\s+", " ", text.replace("\n", " ")).strip()
    return text.strip()

In [9]:
df_clean = df["content"].apply(clean_text)
df_clean.head()

0    I'm also buying more alts while anticipating t...
1    GM chat! ... 🧡 I want to start this day with s...
2    The biggest challenge in bringing TradFi and D...
3                                        $hunt is best
4                                                $Core
Name: content, dtype: object

In [34]:
paragraph = ("".join(df_clean.dropna().astype(str)))

### Sentences Frequency

In [11]:
sentences = re.split(r'(?<=[.!?])\s+', paragraph.strip())
print("Number of sentences =", len(sentences))

Number of sentences = 1715


**Using NLTK Sentence Tokenizer**

In [12]:
sentences = sent_tokenize(paragraph)
print("Number of sentences =", len(sentences))

Number of sentences = 1254


## Word Frequency

In [13]:
words = paragraph.split()
print("Number of words =", len(words))

Number of words = 26895


In [14]:
words = word_tokenize(paragraph)
print("Number of words =", len(words))

Number of words = 34769


### Tokenization

In [15]:
stop_words = set(stopwords.words('english'))
n = 50

In [16]:
def clean_token(token):
    token = re.sub(r"[^\w@$]", "", token)
    return token if token else None

In [17]:
tokens = paragraph.split()
#Clean the token so that @ and $ is allowed 
cleaned_tokens = [clean_token(t) for t in tokens]
#Remove empty tokens None, "" 
cleaned_tokens = [t for t in cleaned_tokens if t]

In [18]:
count=0
unique_tokens = set() 
print("ORIGINAL TOKENS WITH ID")
for i, token in enumerate(cleaned_tokens[:n]):
    word = token.lower()
    if word not in unique_tokens:
        print(i, ":", token)
        unique_tokens.add(word)     

ORIGINAL TOKENS WITH ID
0 : Im
1 : also
2 : buying
3 : more
4 : alts
5 : while
6 : anticipating
7 : the
8 : launch
9 : of
10 : $XOR
11 : With
14 : IDO
15 : now
16 : live
17 : on
18 : @XorionChain
19 : until
20 : 31st
21 : August
22 : its
24 : perfect
25 : time
26 : to
27 : secure
28 : your
29 : allocationGM
30 : chat
31 : I
32 : want
34 : start
35 : this
36 : day
38 : short
39 : plan@MavrykNetwork
40 : working
42 : another
43 : article
44 : about
46 : project
47 : be
48 : ready
49 : for


In [114]:
## Assigning unique_tokens as set since a set cannot contain duplicate values

### Stopword Filtering with Token Iteration

In [19]:
count = 0
for i, w in enumerate(cleaned_tokens[:n]):
    if w.lower() not in stop_words:
        print(i, ":", w)
        count += 1
        if count == 20:
            break

0 : Im
1 : also
2 : buying
4 : alts
6 : anticipating
8 : launch
10 : $XOR
13 : $XOR
14 : IDO
16 : live
18 : @XorionChain
20 : 31st
21 : August
24 : perfect
25 : time
27 : secure
29 : allocationGM
30 : chat
32 : want
34 : start


##### Observation: Stopword filtering removes commonly used words such as - more, while, the, of

In [113]:
# Do we remove the stop words from our cleaned token list?

### Sentiment Analysis

In [20]:
buy = ["buy","buying","long","bullish","calls","moon","dip","breakout","undervalued","accumulate"]
sell = ["sell","selling","short","bearish","puts","dump","crash","overvalued","correction","exit"]

In [21]:
buy_count = 0 
sell_count = 0 
for w in cleaned_tokens:
    word = w.lower()
    if word in buy:
        buy_count +=1 
    elif word in sell:
        sell_count += 1

# Print results
print("Positive words count:", buy_count)
print("Negative words count:", sell_count)

Positive words count: 148
Negative words count: 42


### POS Tagging

In [137]:
# using pos_tag store each tag for a word in a tuple

In [22]:
pos_storage = nltk.pos_tag(cleaned_tokens)
pos_storage

[('Im', 'NNP'),
 ('also', 'RB'),
 ('buying', 'VBG'),
 ('more', 'JJR'),
 ('alts', 'NNS'),
 ('while', 'IN'),
 ('anticipating', 'VBG'),
 ('the', 'DT'),
 ('launch', 'NN'),
 ('of', 'IN'),
 ('$XOR', 'NN'),
 ('With', 'IN'),
 ('the', 'DT'),
 ('$XOR', 'JJ'),
 ('IDO', 'NNP'),
 ('now', 'RB'),
 ('live', 'VBP'),
 ('on', 'IN'),
 ('@XorionChain', 'NN'),
 ('until', 'IN'),
 ('31st', 'CD'),
 ('August', 'NNP'),
 ('its', 'PRP$'),
 ('the', 'DT'),
 ('perfect', 'JJ'),
 ('time', 'NN'),
 ('to', 'TO'),
 ('secure', 'VB'),
 ('your', 'PRP$'),
 ('allocationGM', 'NN'),
 ('chat', 'VBP'),
 ('I', 'PRP'),
 ('want', 'VBP'),
 ('to', 'TO'),
 ('start', 'VB'),
 ('this', 'DT'),
 ('day', 'NN'),
 ('with', 'IN'),
 ('short', 'JJ'),
 ('plan@MavrykNetwork', 'NN'),
 ('working', 'VBG'),
 ('on', 'IN'),
 ('another', 'DT'),
 ('article', 'NN'),
 ('about', 'IN'),
 ('the', 'DT'),
 ('project', 'NN'),
 ('be', 'VB'),
 ('ready', 'JJ'),
 ('for', 'IN'),
 ('alpha', 'JJ'),
 ('@billions_ntwk', 'CD'),
 ('locking', 'VBG'),
 ('in', 'IN'),
 ('the', 'DT

In [23]:
unique_pos = {tag for word, tag in pos_storage}
pos_list = list(unique_pos)
print(pos_list)

['PRP$', 'TO', 'NNS', 'UH', 'VBN', 'MD', 'VB', 'PRP', 'POS', 'VBD', 'VBG', 'RB', 'EX', 'WRB', 'RBS', 'WDT', 'RBR', 'JJ', 'CD', 'CC', 'WP', '$', 'VBZ', 'IN', 'RP', 'JJR', 'NNP', 'PDT', 'NNPS', 'VBP', 'JJS', 'NN', 'DT', 'FW']


In [24]:
pos_counts = Counter(tag for word, tag in pos_storage)
pos_counts

Counter({'NNP': 5106,
         'NN': 4339,
         'IN': 2345,
         'JJ': 2032,
         'DT': 1650,
         'NNS': 1338,
         'RB': 1021,
         'VBZ': 942,
         'CD': 888,
         'VB': 829,
         'VBP': 688,
         'CC': 666,
         'VBG': 661,
         'PRP': 568,
         'TO': 510,
         'VBD': 429,
         'VBN': 337,
         'PRP$': 330,
         'MD': 225,
         'WRB': 136,
         'JJR': 99,
         'RP': 99,
         'WDT': 83,
         'WP': 69,
         'FW': 62,
         'JJS': 53,
         'RBR': 45,
         'EX': 22,
         'NNPS': 22,
         'PDT': 17,
         '$': 13,
         'RBS': 10,
         'UH': 1,
         'POS': 1})

**Filter by noun**

In [25]:
nouns = [word for word, tag in pos_storage if tag.startswith("NN")]
print(nouns)

['Im', 'alts', 'launch', '$XOR', 'IDO', '@XorionChain', 'August', 'time', 'allocationGM', 'day', 'plan@MavrykNetwork', 'article', 'project', 'Billions', 'discord', 'SUPER', 'OG@peaq', 'content', 'Acurast', 'alpha', 'httpstcoypruXCvQkCLets', '@avantisfi', 'gonna', 'join', 'Predator', 'Arena', '$OP@RialoHQ', 'mindshare', 'providing', 'quality', 'challenge', 'TradFi', 'DeFi', 'trust', 'compliance', 'securityThats', 'Haven1', 'blockchain', 'REKT', 'resistant', 'compliant', 'Layer', 'built', 'DeFi', 'reliable', 'everyoneHeres', 'Haven1', 'Control', 'Transparency', 'charge', 'assets', 'custody', 'access', 'network', 'Compliance', 'Compromise', 'time', 'KYC', 'world', 'users', 'institutions', 'assurance', 'NextLevel', 'Security', 'Haven1', 'DeFi', 'problems', 'MEV', 'contract', 'exploits', 'fraud', 'proofs', 'dispute', 'mechanisms', 'safety', 'Real', 'World', 'Connection', 'Native', 'support', 'stablecoins', 'USDCEURC', 'integration', 'payment', 'rails', 'SWIFTSEPA', 'DEXs', 'meet', 'complian

**Filter by verb**

In [26]:
verbs = [word for word, tag in pos_storage if tag.startswith("VB")]
print(verbs)

['buying', 'anticipating', 'live', 'secure', 'chat', 'want', 'start', 'working', 'be', 'locking', 'get', 'working', 'exploring', 'make', 'fun', 'fight', 'building', 'keep', 'contentThe', 'bringing', 'has', 'been', 'comes', 'make', 'open', 'is', 'stay', 'enjoying', 'unlocks', 'interact', 'get', 'need', 'is', 'built', 'stop', 'front', 'running', 'are', 'Works', 'bridging', 'unlocks', 'benefit', 'Governance', 'driven', 'working', 'is', 'building', 'missing', 'make', 'is', 'best$CoreGme', 'has', 'moved', 'are', 'is', 'go', 'is', 'was', 'meant', 'be', 'workThats', 'Decentralized', 'Built', 'Native', 'bridges', 'are', 'moreThis', 'happening', 'is', 'feel', 'momentSending', 'was', 'felt', 'trusted', 'moveThen', 'came', 'missing', 'connects', 'nativelyAnd', 'experience', 'theres', 'going', '00210Targets', 'protecting', 'is', 'forming', 'is', 'pressing', 'looks', 'is', 'has', 'stormed', 'showing', 'staying', 'steals', 'have', 'buzzing', 'are', 'rotating', 'betting', 'heating', 'focused', 'have'

**Group Words by POS Tag**

In [27]:
pos_grouped = defaultdict(list)
for word, tag in pos_storage:
    pos_grouped[tag].append(word)
pos_grouped

defaultdict(list,
            {'NNP': ['Im',
              'IDO',
              'August',
              'Billions',
              'SUPER',
              'OG@peaq',
              'Acurast',
              '@avantisfi',
              'Predator',
              'Arena',
              'TradFi',
              'DeFi',
              'Haven1',
              'REKT',
              'Layer',
              'DeFi',
              'Haven1',
              'Control',
              'Transparency',
              'Compromise',
              'KYC',
              'NextLevel',
              'Security',
              'Haven1',
              'DeFi',
              'MEV',
              'Real',
              'World',
              'Connection',
              'Native',
              'USDCEURC',
              'SWIFTSEPA',
              'DEXs',
              'Liquidity',
              'DeFi',
              'Haven1',
              'Balances',
              'DAO',
              'Decentralization',
              '@hEarn_H

**Converting to data frame**

In [159]:
df_pos = pd.DataFrame(pos_storage, columns=["word", "pos"])
df_pos

,word,pos
0,Im,NNP
1,also,RB
2,buying,VBG
3,more,JJR
4,alts,NNS
...,...,...
25631,pop,VBP
25632,with,IN
25633,gifting,NN
25634,and,CC


In [161]:
filtered = [ (w, t) for w, t in pos_storage if t.startswith(("NN", "VB")) ]
filtered

[('Im', 'NNP'),
 ('buying', 'VBG'),
 ('alts', 'NNS'),
 ('anticipating', 'VBG'),
 ('launch', 'NN'),
 ('$XOR', 'NN'),
 ('IDO', 'NNP'),
 ('live', 'VBP'),
 ('@XorionChain', 'NN'),
 ('August', 'NNP'),
 ('time', 'NN'),
 ('secure', 'VB'),
 ('allocationGM', 'NN'),
 ('chat', 'VBP'),
 ('want', 'VBP'),
 ('start', 'VB'),
 ('day', 'NN'),
 ('plan@MavrykNetwork', 'NN'),
 ('working', 'VBG'),
 ('article', 'NN'),
 ('project', 'NN'),
 ('be', 'VB'),
 ('locking', 'VBG'),
 ('Billions', 'NNP'),
 ('discord', 'NN'),
 ('get', 'VB'),
 ('SUPER', 'NNP'),
 ('OG@peaq', 'NNP'),
 ('working', 'VBG'),
 ('content', 'NN'),
 ('exploring', 'VBG'),
 ('Acurast', 'NNP'),
 ('alpha', 'NN'),
 ('httpstcoypruXCvQkCLets', 'NNS'),
 ('make', 'VBP'),
 ('fun', 'VBZ'),
 ('@avantisfi', 'NNP'),
 ('gonna', 'NN'),
 ('join', 'NN'),
 ('Predator', 'NNP'),
 ('Arena', 'NNP'),
 ('fight', 'VB'),
 ('$OP@RialoHQ', 'NNS'),
 ('building', 'VBG'),
 ('mindshare', 'NN'),
 ('keep', 'VB'),
 ('providing', 'NN'),
 ('quality', 'NN'),
 ('contentThe', 'VBP'),
 ('

**Extracting Sentences with buy / sell keywords**

In [31]:
# Escape keywords in case of special characters
buy_pattern = r"\b(" + "|".join(map(re.escape, buy)) + r")\b"
sell_pattern = r"\b(" + "|".join(map(re.escape, sell)) + r")\b"

buy_regex = re.compile(buy_pattern, re.IGNORECASE)
sell_regex = re.compile(sell_pattern, re.IGNORECASE)

In [32]:
# Split into sentences
sentences = re.split(r'(?<=[.!?])\s+', text.strip())
results = []

def extract_buy_sell_sentences(text):
    # Split into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    results = []
    for s in sentences:
        has_buy = bool(buy_regex.search(s))
        has_sell = bool(sell_regex.search(s))

        if has_buy or has_sell:
            label = "both" if has_buy and has_sell else "buy" if has_buy else "sell"
            results.append({"sentence": s,"label": label,"has_buy": has_buy,"has_sell": has_sell})
    return results

In [33]:
text = """
I think we should buy AAPL before earnings.
Traders are bearish on TSLA.
It might be time to sell NVDA if support breaks.
Strong buy signal forming on MSFT.
"""

matches = extract_buy_sell_sentences(text)

for m in matches:
    print(m)

{'sentence': 'I think we should buy AAPL before earnings.', 'label': 'buy', 'has_buy': True, 'has_sell': False}
{'sentence': 'Traders are bearish on TSLA.', 'label': 'sell', 'has_buy': False, 'has_sell': True}
{'sentence': 'It might be time to sell NVDA if support breaks.', 'label': 'sell', 'has_buy': False, 'has_sell': True}
{'sentence': 'Strong buy signal forming on MSFT.', 'label': 'buy', 'has_buy': True, 'has_sell': False}
